# Tutorial 3 Part B Collision Avoidance

This tutorial will show you how to use a captured depth image to avoid collisions. A basic understanding of Python and NumPy arrays is required to implement image processing techniques. 

The depth image provides distance information about the scene. Collision avoidance can be achieved by calculating the shortest distance directly in front of the robot. Since the camera is fixed at the front, we can approximate the robot's frontal area as the central region of the captured image. The following image, with a resolution of 672×376, illustrates the defined central area:

![TEST](pixel.png)

To focus on the central area, we can first set the values of the surrounding area to 0 and then remove them later by considering only the non-zero regions. (Note: When implementing your algorithm in Python, avoid using for loops, as they are very slow.) The following code demonstrates how to set 0 values in an array named depth_image.

                depth_image[:94,:] = 0
                depth_image[282:,:]=0
                depth_image[:,:168]=0
                depth_image[:,504:]=0

Remember that in Python, the indexing convention for an array follows the format array[row, column]. The coordinates provided above are just an example. In practice, you may need to adjust these values to achieve better performance.

Now that we have extracted the central area, we only need to find the minimum value within this region to perform object avoidance. Keep in mind that sensor readings may contain noise. We can easily filter out these noises, along with some distant objects, using the following code:

                depth_image[depth_image<100]=0
                depth_image[depth_image>1000]=0

Normally, we could use a for loop to iterate over each pixel in depth_image and compare its value to a threshold. However, this approach significantly impacts processing speed, as Python is inefficient when handling array operations using loops. To improve performance, it is recommended to use built-in array operations whenever possible.

For example, NumPy allows direct comparison between an array and a number using the statement depth_image < 100. This is an element-wise operation that compares each pixel's value to the threshold and returns a Boolean array of True or False values. We can then use this Boolean array to index depth_image and set the True areas to 0.

Below is a link to learn more about NumPy operations:
https://numpy.org/devdocs/user/absolute_beginners.html


Then, the minimum value inside an array can be found using the following code:

                depth_image[depth_image!=0].min()

However, the code above has a small issue: if all values in depth_image are 0, this command will result in an error. Check the following code to see how to fix this bug. Try running it and explore further improvements.

In [26]:
camera.stop()
robot.stop()

In [3]:
#create two widgets for the displaying of the image
import ipywidgets.widgets as widgets
from IPython.display import display, HTML
import sys

#create widgets for the displaying of the image
display_color = widgets.Image(format='jpeg', width='30%') #determine the width of the color image
display_depth = widgets.Image(format='jpeg', width='30%')  #determine the width of the depth image
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget

import cv2
import numpy as np
import pyzed.sl as sl
import math
import numpy as np
import sys
import math

#multi-threading is used to capture the image in real time performance
import threading

import time
import motors

#initialize the Robot class
robot = motors.MotorsYukon(mecanum=False)

class Camera():
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        # Create a InitParameters object and set configuration parameters
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA #VGA(672*376), HD720(1280*720), HD1080 (1920*1080) or ...
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # Use ULTRA depth mode
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Use meter units (for depth measurements)

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS: #Ensure the camera has opened succesfully
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

         # Create and set RuntimeParameters after opening the camera
        self.runtime = sl.RuntimeParameters()

        #flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width,self.height,sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C4, sl.MEM.CPU) 

    def _capture_frames(self):

        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
           
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value = self.image.get_data()
                #When displaying the images, the robot sends data to the computers. If the images are in very high resolution, it may cause network issues. Since the images are for display purposes only, please use a lower resolution if possible. Thank you.
                scale = 0.1
                resized_image = cv2.resize(self.color_value, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
                cv2.circle(resized_image, (int(self.width*scale//2),int(self.height*scale//2)), 1, (0, 255, 0))
                display_color.value = bgr8_to_jpeg(resized_image)
                
                self.depth_image = np.asanyarray(self.depth.get_data())

                depth_image_test = self.depth_image.copy()  #copy the depth image
                depth_image_test = np.nan_to_num(depth_image_test, nan=0.0).astype(np.float32)  # Change NaN value to 0
                depth_image_test[:94,:] = 0
                depth_image_test[282:,:]=0
                depth_image_test[:,:168]=0
                depth_image_test[:,504:]=0

                depth_image_test[depth_image_test<200]=0
                depth_image_test[depth_image_test>1000]=0
                depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image_test, alpha=0.03), cv2.COLORMAP_JET)
                if(depth_image_test.max() == 0):
                    robot.backward(1) # Turn back
                    cv2.putText(depth_colormap, 'warning!!!', (336,188), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                else:
                    left_side=depth_image_test[:, :336]
                    left_side_distance = left_side[left_side!=0].min()
                    print("left" + str(left_side_distance))
                    right_side=depth_image_test[:, 336:]
                    print(right_side)
                    right_side_distance = right_side[right_side!=0].min()
                    print(right_side_distance)
                    print("right" + str(right_side_distance))
                    
                    if (left_side_distance < right_side_distance and left_side_distance < 400):
                        robot.right(0.7)
                        cv2.putText(depth_colormap, 'warning!!!', (336,188), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        time.sleep(0.1)
                    elif(right_side_distance < left_side_distance and right_side_distance < 400):
                        robot.left(0.7) #turn left at full speed
                        cv2.putText(depth_colormap, 'warning!!!', (336,188), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
                        time.sleep(0.1)
                    else:
                        robot.forward(0.4) #move forward at half speed
                    
                resized_depth_colormap=cv2.resize(depth_colormap, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
                display_depth.value = bgr8_to_jpeg(resized_depth_colormap)
                
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread
            self.zed.close()

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera()
camera.start() # start capturing the data


[2025-02-19 10:53:12 UTC][ZED][INFO] Logging level INFO
[2025-02-19 10:53:12 UTC][ZED][INFO] Logging level INFO
[2025-02-19 10:53:12 UTC][ZED][INFO] Logging level INFO
[2025-02-19 10:53:12 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-02-19 10:53:13 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-02-19 10:53:13 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-02-19 10:53:13 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-02-19 10:53:13 UTC][ZED][INFO] [Init]  Serial Number: S/N 37297852
left609.5304
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
619.28595
right619.28595
left587.4444
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
751.983
right751.983
left563.5138
[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. .

In [2]:
camera.stop()
robot.stop()

**Tasks**

1.	Understand how collision avoidance can be achieved.

2.	Different robots have different viewing angles, so the predefined area might not work best for your robot. Please adjust the area to see if the performance can be improved.

3.	Please adjust the distance threshold and center region to achieve the best performance  

4.	In the demo code, the robot will always turn left after detecting the collisions. Are there better strategies to handle this?

Optional: Check whether the program can run while simultaneously recording data.

Note: Please remember to delete your code on the robot once the experiment is finished.